In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197,5호선 상일동,373.362877,NaN
1,12,28480888,옥돌영양탕,NaN,Q,음식,Q05,닭/오리요리,Q05A01,삼계탕전문,...,143220,4945.0,NaN,1,NaN,127.089673,37.557977,5호선 아차산(어린이대공원후문),354.921957,NaN
2,14,17175285,은진이네,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,121861,4206.0,NaN,NaN,NaN,126.956164,37.551340,5호선 애오개,53.083100,NaN
3,15,17175333,흥부보쌈,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,158721,7997.0,NaN,5,NaN,126.875987,37.529132,5호선 오목교(목동운동장앞),353.540138,NaN
4,28,20665661,가파도참치,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,158050,8005.0,NaN,2,NaN,126.873242,37.524340,5호선 오목교(목동운동장앞),122.836352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,135800,17212093,만랩커피,오금역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138800,5702.0,NaN,NaN,103,127.125353,37.502381,5호선 오금,124.273859,NaN
20045,135805,17218896,얼음과자,NaN,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,157857,7510.0,NaN,1,123,126.812418,37.577436,5호선 방화,27.430119,NaN
20046,135806,17218871,오이소서울,상일점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134090,5276.0,NaN,1,104,127.165574,37.550924,5호선 상일동,983.009054,NaN
20047,135807,17218873,얼음왕국명일14,6호점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134780,5297.0,상가,1,106,127.146217,37.546317,5호선 굽은다리(강동구민회관앞),347.273163,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,NaN,상일동,상일동역 도전최강달인왕만두
1,옥돌영양탕,닭/오리요리,삼계탕전문,서울특별시 광진구 긴고랑로36길 24,5호선 아차산(어린이대공원후문),354.921957,NaN,아차산,아차산역 옥돌영양탕
2,은진이네,한식,한식/백반/한정식,서울특별시 마포구 마포대로 196,5호선 애오개,53.083100,NaN,애오개,애오개역 은진이네
3,흥부보쌈,한식,족발/보쌈전문,서울특별시 양천구 목동동로 309,5호선 오목교(목동운동장앞),353.540138,NaN,오목교,오목교역 흥부보쌈
4,가파도참치,일식/수산물,참치전문점,서울특별시 양천구 오목로54길 2,5호선 오목교(목동운동장앞),122.836352,NaN,오목교,오목교역 가파도참치
...,...,...,...,...,...,...,...,...,...
20044,만랩커피,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 중대로23길 21,5호선 오금,124.273859,NaN,오금,오금역 만랩커피
20045,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,NaN,방화,방화역 얼음과자
20046,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,NaN,상일동,상일동역 오이소서울
20047,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,NaN,굽은다리,굽은다리역 얼음왕국명일14


In [5]:
items = data["검색어"][:7]
items

0    상일동역 도전최강달인왕만두
1        아차산역 옥돌영양탕
2         애오개역 은진이네
3         오목교역 흥부보쌈
4        오목교역 가파도참치
5         방화역 울엄니밥상
6           명일역 아이포
Name: 검색어, dtype: object

In [6]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [148]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img ') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img ') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].text}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")


진행상황 : 1/20049
식당 존재 x
진행상황 : 2/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/20049
식당 존재 x
진행상황 : 6/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/20049
식당 존재 x
진행상황 : 9/20049
식당 존재 x
진행상황 : 10/20049
식당 존재 x
진행상황 : 11/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/20049
식당 존재 x
진행상황 : 14/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/20049
식당 존재 x
진행상황 : 17/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 18/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 19/20049
식당 존재 x
진행상황 : 20/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 21/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 22/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 23/20049
식당 존재 x
진행상황 : 24/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 186/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 187/20049
식당 존재 x
진행상황 : 188/20049
식당 존재 x
진행상황 : 189/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 190/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 191/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 192/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 193/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 194/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 195/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 196/20049
식당 존재 x
진행상황 : 197/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 198/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 199/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 200/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 201/20049
식당 존재 x
진행상황 : 202/20049
식당 존재 x
진행상황 : 203/20049
식당 존재 x
진행상황 : 204/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 205/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 206/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 207/20049
식당 존재
5
더 이상 리뷰

식당 존재 x
진행상황 : 390/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 392/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 393/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 394/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 395/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 396/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 397/20049
식당 존재 x
진행상황 : 398/20049
식당 존재 x
진행상황 : 399/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 400/20049
식당 존재 x
진행상황 : 401/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 402/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 403/20049
식당 존재 x
진행상황 : 404/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 405/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 406/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 407/20049
식당 존재 x
진행상황 : 408/20049
식당 존재 x
진행상황 : 409/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 410/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 411/200

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 588/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 589/20049
식당 존재 x
진행상황 : 590/20049
식당 존재 x
진행상황 : 591/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 592/20049
식당 존재 x
진행상황 : 593/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 594/20049
식당 존재 x
진행상황 : 595/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 596/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 597/20049
식당 존재 x
진행상황 : 598/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 599/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 600/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 601/20049
식당 존재 x
진행상황 : 602/20049
식당 존재 x
진행상황 : 603/20049
식당 존재 x
진행상황 : 604/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 605/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 606/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 607/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 608/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 609/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 610/20049
식당 존재 x
진행상황 : 61

식당 존재 x
진행상황 : 778/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 779/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 780/20049
식당 존재 x
진행상황 : 781/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 782/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 783/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 784/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 785/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 786/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 787/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 788/20049
식당 존재 x
진행상황 : 789/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 790/20049
식당 존재 x
진행상황 : 791/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 792/20049
식당 존재 x
진행상황 : 793/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 794/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 795/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 796/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 797/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 798/20049
식당 존재
4
더 이상 리뷰 존재 X
기본

...

식당 존재 x
진행상황 : 1515/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1516/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1517/20049
식당 존재 x
진행상황 : 1518/20049
식당 존재 x
진행상황 : 1519/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1520/20049
식당 존재 x
진행상황 : 1521/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1522/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1523/20049
식당 존재 x
진행상황 : 1524/20049
식당 존재 x
진행상황 : 1525/20049
식당 존재 x
진행상황 : 1526/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1527/20049
식당 존재 x
진행상황 : 1528/20049
식당 존재 x
진행상황 : 1529/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1530/20049
식당 존재 x
진행상황 : 1531/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1532/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1533/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1534/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1535/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1536/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1537/20049
식

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1701/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1702/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1703/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1704/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1705/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1706/20049
식당 존재 x
진행상황 : 1707/20049
식당 존재 x
진행상황 : 1708/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1709/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1710/20049
식당 존재 x
진행상황 : 1711/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1712/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1713/20049
식당 존재 x
진행상황 : 1714/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1715/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1716/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1717/20049
식당 존재 x
진행상황 : 1718/20049
식당 존재 x
진행상황 : 1719/20049
식당 존재 x
진행상황 : 1720/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1721/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1722/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로

식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1882/20049
식당 존재 x
진행상황 : 1883/20049
식당 존재 x
진행상황 : 1884/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1885/20049
식당 존재 x
진행상황 : 1886/20049
식당 존재 x
진행상황 : 1887/20049
식당 존재 x
진행상황 : 1888/20049
식당 존재 x
진행상황 : 1889/20049
식당 존재 x
진행상황 : 1890/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1891/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1892/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1893/20049
식당 존재 x
진행상황 : 1894/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1895/20049
식당 존재 x
진행상황 : 1896/20049
식당 존재 x
진행상황 : 1897/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1898/20049
식당 존재 x
진행상황 : 1899/20049
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1900/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1901/20049
식당 존재 x
진행상황 : 1902/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1903/20049
식당 존재 x
진행상황 : 1904/20049
식당 존재 x
진행상황 : 1905/20049
식당 존재 x
진행상황 : 1906/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1907/20049
식당

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2073/20049
식당 존재 x
진행상황 : 2074/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2075/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2076/20049
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2077/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2078/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2079/20049
식당 존재 x
진행상황 : 2080/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2081/20049
식당 존재 x
진행상황 : 2082/20049
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2083/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2084/20049
식당 존재 x
진행상황 : 2085/20049
식당 존재 x
진행상황 : 2086/20049
식당 존재 x
진행상황 : 2087/20049
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2088/20049
식당 존재 x
진행상황 : 2089/20049
식당 존재 x
진행상황 : 2090/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2091/20049
식당 존재 x
진행상황 : 2092/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2093/20049
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2094/20049
식당 존재 x
진행상황 : 2095/20049
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2096/

KeyboardInterrupt: 

In [149]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,m6b2hb,방화역 울엄니밥상,1점,2020.10.29.
1,f91t49,방화역 울엄니밥상,5점,2020.10.13.
2,mvub1p,방화역 울엄니밥상,3점,2020.03.06.
3,vrmpok,방화역 울엄니밥상,5점,2019.12.12.
4,7obarf,방화역 울엄니밥상,5점,2017.12.09.
...,...,...,...,...
8208,10jgadh,마포역 도화동지니,5점,2021.03.04.
8209,n2cj41,마포역 도화동지니,3점,2021.01.27.
8210,e1t7fd,마포역 도화동지니,4점,2020.09.30.
8211,bkki64,마포역 도화동지니,5점,2020.07.11.


In [150]:
img_elem

,ItemID,img_url
0,방화역 울엄니밥상,None
1,방화역 울엄니밥상,None
2,방화역 울엄니밥상,None
3,방화역 울엄니밥상,None
4,방화역 울엄니밥상,None
...,...,...
8208,마포역 도화동지니,None
8209,마포역 도화동지니,
8210,마포역 도화동지니,None
8211,마포역 도화동지니,None


In [151]:
review_elem

,ItemID,review
0,방화역 울엄니밥상,음...하나를 보면 열을 안다.
1,방화역 울엄니밥상,고등어구이가 맛있음!!! 후식으로 먹은 식혜도 꿀맛!!
2,방화역 울엄니밥상,보통이에요
3,방화역 울엄니밥상,푸짐하게 나오고 아주머니도 친절했어요
4,방화역 울엄니밥상,방화사거리 최고의 백반집. 백반은 자고로 반찬맛인데. 반찬이 정갈하니 질리지 않고 ...
...,...,...
8208,마포역 도화동지니,진짜 맛있어요 !!!
8209,마포역 도화동지니,스테이크 맛있다 해서 갔는데 퍽퍽하고 질겼어요 ㅜㅜ 사장님과 직원분들은 친절하심
8210,마포역 도화동지니,싼마이에 갈만해요
8211,마포역 도화동지니,최고예요


In [154]:
img_elem.head()

,ItemID,img_url
0,방화역 울엄니밥상,None
1,방화역 울엄니밥상,None
2,방화역 울엄니밥상,None
3,방화역 울엄니밥상,None
4,방화역 울엄니밥상,None


In [164]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID     ItemID Rating    Timestamp
8208  10jgadh  마포역 도화동지니     5점  2021.03.04.
8209   n2cj41  마포역 도화동지니     3점  2021.01.27.
8210   e1t7fd  마포역 도화동지니     4점  2020.09.30.
8211   bkki64  마포역 도화동지니     5점  2020.07.11.
8212   fjtuhl  마포역 도화동지니     4점  2019.11.26.
         ItemID img_url
8208  마포역 도화동지니    None
8209  마포역 도화동지니        
8210  마포역 도화동지니    None
8211  마포역 도화동지니    None
8212  마포역 도화동지니    None
         ItemID                                        review
8208  마포역 도화동지니                                   진짜 맛있어요 !!!
8209  마포역 도화동지니  스테이크 맛있다 해서 갔는데 퍽퍽하고 질겼어요 ㅜㅜ 사장님과 직원분들은 친절하심
8210  마포역 도화동지니                                     싼마이에 갈만해요
8211  마포역 도화동지니                                          최고예요
8212  마포역 도화동지니                                           좋아요


In [155]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_rating(2160).csv')

In [165]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_User_ID(2160).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_img(2160).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_review(2160).csv')

In [89]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


1013

In [100]:
print (data.loc[data['검색어'] == "서대문역 아웃백스테이크하우스"])

             상호명 상권업종중분류명 상권업종소분류명             도로명주소       역명          거리  \
2527  아웃백스테이크하우스       양식  패밀리레스토랑  서울특별시 중구 통일로 114  5호선 서대문  122.856647   

      카카오url    역              검색어  
2527     NaN  서대문  서대문역 아웃백스테이크하우스  


IndexError: list index out of range